In [10]:
# Configurações globais do pandas para não truncar dados nos resultados

import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)


In [1]:
"""
CHUNKING ADAPTATIVO PROFISSIONAL PARA ARTIGOS CIENTÍFICOS
========================================================

Objetivo:
- Criar chunks semânticos de alta qualidade
- Otimizar Recall@k para queries em artigos científicos
- NÃO usar LLM (rápido, reprodutível, escalável)
- Respeitar limites de tamanho (<= 1000 caracteres)
- Preservar estrutura científica (seções, parágrafos)

Pipeline:
1) Limpeza básica do texto
2) Detecção de seções científicas
3) Split em parágrafos
4) Agrupamento adaptativo por tamanho
5) Overlap semântico por parágrafo
6) Geração de Documents prontos para RAG

Python recomendado: 3.11
"""

# ============================================================
# 1. IMPORTS
# ============================================================

import re
from typing import List

from langchain_core.documents import Document


# ============================================================
# 2. CONFIGURAÇÕES GLOBAIS
# ============================================================

# Limites de tamanho
CHUNK_TARGET_SIZE = 800     # alvo ideal
CHUNK_MAX_SIZE = 1000       # limite rígido

# Overlap semântico (em parágrafos)
OVERLAP_PARAGRAPHS = 1

# Cabeçalhos científicos comuns
SECTION_HEADERS = [
    "abstract",
    "introduction",
    "background",
    "related work",
    "related works",
    "available datasets",
    "methodology",
    "proposed methods",
    "methods",
    "materials and methods",
    "experiments",
    "results",
    "discussion",
    "results and discussions",
    "conclusion",
    "conclusions",
    "future work",
    "references",
]


# ============================================================
# 3. LIMPEZA BÁSICA DO TEXTO
# ============================================================

def normalize_text(text: str) -> str:
    """
    Normaliza texto extraído de PDF:
    - Remove espaços excessivos
    - Corrige quebras estranhas
    """
    text = re.sub(r"\n{3,}", "\n\n", text)
    text = re.sub(r"[ \t]+", " ", text)
    return text.strip()


# ============================================================
# 4. DETECÇÃO DE SEÇÕES
# ============================================================

def split_by_sections(text: str) -> List[dict]:
    """
    Divide o texto em seções científicas detectadas via regex.

    Retorna:
    [
        {
            "section": "introduction",
            "content": "texto da seção"
        },
        ...
    ]
    """
    pattern = r"(?i)\n(" + "|".join(SECTION_HEADERS) + r")\n"
    parts = re.split(pattern, text)

    sections = []
    current_section = "unknown"

    for part in parts:
        part_clean = part.strip()

        if part_clean.lower() in SECTION_HEADERS:
            current_section = part_clean.lower()
        elif part_clean:
            sections.append(
                {
                    "section": current_section,
                    "content": part_clean,
                }
            )

    return sections


# ============================================================
# 5. SPLIT EM PARÁGRAFOS
# ============================================================

def split_into_paragraphs(text: str) -> List[str]:
    """
    Divide texto em parágrafos válidos.
    Ignora parágrafos muito curtos.
    """
    paragraphs = [
        p.strip()
        for p in text.split("\n\n")
        if len(p.strip()) >= 50
    ]
    return paragraphs


# ============================================================
# 6. AGRUPAMENTO ADAPTATIVO DE PARÁGRAFOS
# ============================================================

def adaptive_group_paragraphs(
    paragraphs: List[str],
    target_size: int = CHUNK_TARGET_SIZE,
    max_size: int = CHUNK_MAX_SIZE,
) -> List[str]:
    """
    Agrupa parágrafos respeitando:
    - Não quebrar parágrafo
    - Alvo de tamanho
    - Limite máximo rígido
    """
    chunks = []
    current_chunk = ""

    for paragraph in paragraphs:
        candidate = (
            current_chunk + "\n\n" + paragraph
            if current_chunk
            else paragraph
        )

        if len(candidate) <= max_size:
            current_chunk = candidate
        else:
            if current_chunk:
                chunks.append(current_chunk)
            current_chunk = paragraph

    if current_chunk:
        chunks.append(current_chunk)

    return chunks


# ============================================================
# 7. OVERLAP SEMÂNTICO POR PARÁGRAFO
# ============================================================

def apply_paragraph_overlap(
    chunks: List[str],
    overlap_paragraphs: int = OVERLAP_PARAGRAPHS,
) -> List[str]:
    """
    Aplica overlap reutilizando os últimos parágrafos
    do chunk anterior.
    """
    if overlap_paragraphs <= 0:
        return chunks

    overlapped_chunks = []

    for i, chunk in enumerate(chunks):
        if i == 0:
            overlapped_chunks.append(chunk)
            continue

        prev_paragraphs = chunks[i - 1].split("\n\n")
        overlap = "\n\n".join(prev_paragraphs[-overlap_paragraphs:])

        combined = overlap + "\n\n" + chunk
        overlapped_chunks.append(combined)

    return overlapped_chunks


# ============================================================
# 8. PIPELINE PRINCIPAL DE CHUNKING ADAPTATIVO
# ============================================================

def adaptive_chunking(text: str, source_name: str) -> List[Document]:
    """
    Executa o pipeline completo de chunking adaptativo
    e retorna Documents prontos para RAG.
    """
    text = normalize_text(text)
    sections = split_by_sections(text)

    documents = []
    global_chunk_id = 0

    for section_data in sections:
        section_name = section_data["section"]
        section_text = section_data["content"]

        paragraphs = split_into_paragraphs(section_text)
        base_chunks = adaptive_group_paragraphs(paragraphs)
        final_chunks = apply_paragraph_overlap(base_chunks)

        for chunk in final_chunks:
            documents.append(
                Document(
                    page_content=chunk,
                    metadata={
                        "source": source_name,
                        "section": section_name,
                        "chunk_id": global_chunk_id,
                        "chunking": "adaptive",
                        "length": len(chunk),
                    },
                )
            )
            global_chunk_id += 1

    return documents


# ============================================================
# 9. EXEMPLO DE USO
# ============================================================

"""
text = load_pdf("artigo_cientifico.pdf")

documents = adaptive_chunking(
    text=text,
    source_name="artigo_cientifico.pdf"
)

print(f"Total de chunks gerados: {len(documents)}")

# Visualizar um chunk
print(documents[0].metadata)
print(documents[0].page_content[:500])
"""


# ============================================================
# 10. CONCLUSÃO PROFISSIONAL
# ============================================================

"""
Este chunking adaptativo é:
✔ Rápido (sem LLM)
✔ Escalável
✔ Ideal para artigos científicos
✔ Otimizado para Recall@k
✔ Padrão de produção em RAG sério

Você pode agora:
- Indexar no FAISS
- Avaliar Recall@k
- Ajustar target_size empiricamente
"""


'\nEste chunking adaptativo é:\n✔ Rápido (sem LLM)\n✔ Escalável\n✔ Ideal para artigos científicos\n✔ Otimizado para Recall@k\n✔ Padrão de produção em RAG sério\n\nVocê pode agora:\n- Indexar no FAISS\n- Avaliar Recall@k\n- Ajustar target_size empiricamente\n'

In [ ]:
import os

from pypdf import PdfReader
from docx import Document as DocxDocument

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_ollama import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

def load_pdf(path: str) -> str:
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text


text = load_pdf("datasets/tattoo.pdf")

documents = adaptive_chunking(
    text=text,
    source_name="tattoo.pdf"
)



Total de chunks gerados: 6
{'source': 'tattoo.pdf', 'section': 'unknown', 'chunk_id': 0, 'chunking': 'adaptive', 'length': 11933}
Date of publication xxxx 00, 0000, date of current version xxxx 00, 0000.
Digital Object Identifier 10.1109/ACCESS.2017.DOI
Open-Set Tattoo Semantic Segmentation
ANDERSON BRILHADOR 1, RODRIGO TCHALSKI DA SIL VA 1, CARLOS ROBERTO
MODINEZ-JUNIOR 1, GABRIEL DE ALMEIDA SP ADAFORA 1, HEITOR SIL VÉRIO
LOPES 1, AND ANDRÉ EUGÊNIO LAZZARETTI 1, (Member, IEEE).
1Federal University of Technology - Paraná, Av. Sete de Setembro, 3165, Curitiba, 80230-901, Paraná, Brazil.
Corresponding author: Anderson Brilhador (e-mail: andersonbrilhador@gmail.com).
ABSTRACT Tattoos can serve as an essential source of biometric information for public security, aiding
in identifying suspects and victims. In order to automate tattoo classification, tasks like classification
require more detailed image content analysis, such as semantic segmentation. However, a dataset with
appropriate sema

In [12]:
# ============================================================
# ITERAR POR TODOS OS CHUNKS GERADOS
# ============================================================

for i, doc in enumerate(documents):
    print("=" * 80)
    print(f"Chunk global index: {i}")
    print("Metadata:")
    for k, v in doc.metadata.items():
        print(f"  {k}: {v}")

    print("-" * 80)
    print("Conteúdo (início do chunk):")
    print(doc.page_content[:500])  # limita para não poluir o terminal
    print("\n")

Chunk global index: 0
Metadata:
  source: tattoo.pdf
  section: unknown
  chunk_id: 0
  chunking: adaptive
  length: 11933
--------------------------------------------------------------------------------
Conteúdo (início do chunk):
Date of publication xxxx 00, 0000, date of current version xxxx 00, 0000.
Digital Object Identifier 10.1109/ACCESS.2017.DOI
Open-Set Tattoo Semantic Segmentation
ANDERSON BRILHADOR 1, RODRIGO TCHALSKI DA SIL VA 1, CARLOS ROBERTO
MODINEZ-JUNIOR 1, GABRIEL DE ALMEIDA SP ADAFORA 1, HEITOR SIL VÉRIO
LOPES 1, AND ANDRÉ EUGÊNIO LAZZARETTI 1, (Member, IEEE).
1Federal University of Technology - Paraná, Av. Sete de Setembro, 3165, Curitiba, 80230-901, Paraná, Brazil.
Corresponding author: Anderson Brilha


Chunk global index: 1
Metadata:
  source: tattoo.pdf
  section: background
  chunk_id: 1
  chunking: adaptive
  length: 145
--------------------------------------------------------------------------------
Conteúdo (início do chunk):
Flower
Leaf
Stem
Unknown 
classe

In [13]:
# ============================================================
# ITERAR APENAS CHUNKS DE UMA SEÇÃO ESPECÍFICA
# ============================================================

target_section = "abstract"

for doc in documents:
    if doc.metadata.get("section") == target_section:
        print("=" * 80)
        print(f"Section: {target_section}")
        print(f"Chunk ID: {doc.metadata['chunk_id']}")
        print("-" * 80)
        print(doc.page_content[:500])
        print()


In [14]:
# ============================================================
# INSPEÇÃO RÁPIDA DE QUALIDADE DOS CHUNKS
# ============================================================

lengths = [len(doc.page_content) for doc in documents]

print(f"Total de chunks: {len(lengths)}")
print(f"Tamanho médio: {sum(lengths) / len(lengths):.1f}")
print(f"Menor chunk: {min(lengths)}")
print(f"Maior chunk: {max(lengths)}")


Total de chunks: 6
Tamanho médio: 18399.2
Menor chunk: 108
Maior chunk: 56269
